#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


684


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

566


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Let's create a keyspace to do our work in 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Connect to our Keyspace. Compare this to how we had to create a new session in PostgreSQL.  
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


In [8]:
# Table for Query 1 (Songs by Session)
query = "CREATE TABLE IF NOT EXISTS song_session "
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length decimal, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
# Table for Query 2 (Songs by User & Session)
query = "CREATE TABLE IF NOT EXISTS song_user_session "
query = query + "(user_id int, session_id int, song_title text, item_in_session int, artist text, first_name text, last_name text, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Table for Query 3 (Users by Song Title)
query = "CREATE TABLE IF NOT EXISTS user_song_title "
query = query + "(song_title text, artist text, first_name text, last_name text, PRIMARY KEY (song_title, artist, last_name, first_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
song_session_insert = "INSERT INTO song_session (session_id, item_in_session, artist, song_title, song_length) "
song_session_insert = song_session_insert + " VALUES (%s, %s, %s, %s, %s)"

song_user_session_insert = "INSERT INTO song_user_session (user_id, session_id, song_title, item_in_session, artist, first_name, last_name) "
song_user_session_insert = song_user_session_insert + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

user_song_title_insert = "INSERT INTO user_song_title (song_title, artist, first_name, last_name) "
user_song_title_insert = user_song_title_insert + " VALUES (%s, %s, %s, %s)"

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT statements
#     session_song (session_id, item_in_session, artist, song_title, song_length)
        session.execute(song_session_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
#     user_song (user_id, session_id, song_title, item_in_session, artist, first_name, last_name)
        session.execute(song_user_session_insert, (int(line[10]), int(line[8]), line[9], int(line[3]), line[0], line[1], line[4]))
#     song_title_user (song_title, artist, first_name, last_name)
        session.execute(user_song_title_insert, (line[9], line[0], line[1], line[4]))

#### Running a SELECT to verify that the data have been inserted into each table

In [11]:
query = "SELECT artist, song_title, song_length FROM song_session WHERE session_id=338 AND item_in_session=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.song_length)
                    

In [12]:
query = "SELECT artist, song_title, first_name, last_name FROM song_user_session WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [13]:
query = "SELECT first_name, last_name FROM user_song_title WHERE song_title='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)

                    

Sara Johnson


### Drop the tables before closing out the sessions

In [14]:
query = "DROP TABLE IF EXISTS song_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS song_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS user_song_title"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()